In [1]:
import pandas as pd
import numpy
import requests
import acquire
import os
from sklearn.model_selection import train_test_split

In [2]:
df = acquire.get_competition_data()

In [3]:
df.head()

,Unnamed: 0,Skater,Nat_x,SP,SP.1,SP.2,FS,FS.1,FS.2,Total,...,PCS.1_y,TSS_y,season,event,QA,QA.1,QB,QB.1,PR,PR.1
0,1,Arakawa,🇯🇵,PB,66.02,3.0,PB,125.32,1.0,PB,...,1,-0.00125.32,2006,olympics,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Cohen,🇺🇸,SB,66.73,1.0,SB,116.63,2.0,SB,...,2,-1.00116.63,2006,olympics,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Slutskaya,🇷🇺,NaN,66.70,2.0,NaN,114.74,3.0,NaN,...,3,-1.00114.74,2006,olympics,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Suguri,🇯🇵,SB,61.75,4.0,SB,113.48,4.0,SB,...,4,-0.00113.48,2006,olympics,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Rochette,🇨🇦,NaN,55.85,9.0,SB,111.42,5.0,SB,...,5,-0.00111.42,2006,olympics,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.info()

In [ ]:
#steps for prepare

In [4]:
df = df.drop(columns=['QA', 'QA.1', 'QB', 'QB.1', 'PR', 'PR.1', 'Nat_x', 'Nat_y', 'season_x', '#_x', '#_y'])
df['SP'] = df.SP.fillna('no_best')
df['FS'] = df.FS.fillna('no_best')

In [ ]:
#df[['skater_name','event','Unnamed: 0']]

In [5]:
df['Solo Jump'] = df['Solo Jump'].fillna('3Lz')

In [6]:
df['Total'] = df['Total'].fillna('no_best')

In [7]:
df = df.dropna()

In [ ]:
#df = df.drop(columns=['Nat_x', 'Nat_y', 'season_x', '#_x', '#_y', 'TSS_x', 'TSS_y'])

In [8]:
df.rename(columns={'Skater':'last_name', 'SP': 'short_best', 'SP.1': 'short_score', 'SP.2': 'short_place', 'Total': 'event_best', 'Total.1': 'event_score', 'Unnamed: 0': 'event_place',  'Combo Jump': 'short_combo_jump', 'Solo Jump': 'short_solo_jump', 'Axel': 'short_axel_jump', 'TES_x': 'short_elements_score', 'TES.1_x': 'short_elements_rank', 'PCS_x': 'short_components_score', 'PCS.1_x': 'short_components_rank', 'TES_y': 'free_elements_score', 'TES.1_y': 'free_elements_rank', 'PCS_y': 'free_components_score', 'PCS.1_y': 'free_components_rank', 'FS': 'free_best', 'FS.1': 'free_score', 'FS.2': 'free_place', 'Nat': 'country', 'Elements': 'free_elements', 'TSS_x': 'short_overall', 'TSS_y': 'free_overall'}, inplace=True)

In [ ]:
#'FS': 'free_best', 'FS.1': 'free_score', 'FS.2': 'free_place', 'Nat': 'country', 'Elements': 'free_elements'

In [ ]:
df.info()

In [9]:
olympic_skaters = df.skater_name[df.event == 'olympics'].unique()
df['olympian'] = df.apply(lambda x: True if x.skater_name in olympic_skaters else False, axis=1)

In [83]:
df_oly = df[df['olympian'] == True]

In [84]:
df_oly

,event_place,last_name,short_best,short_score,short_place,free_best,free_score,free_place,event_best,event_score,...,country,free_elements,free_elements_score,free_elements_rank,free_components_score,free_components_rank,free_overall,season,event,olympian
0,1,Arakawa,PB,66.02,3.0,PB,125.32,1.0,PB,191.34,...,🇯🇵,"3Lz+2Lo , 3S+2T , 3F , 2A , 3Lzx , 2Lox , 3S+2...",62.32,1,63.00,1,-0.00125.32,2006,olympics,True
1,2,Cohen,SB,66.73,1.0,SB,116.63,2.0,SB,183.36,...,🇺🇸,"3Lz , 3F , 3Lo , 3F+2Tx , 3T+3S+SEQx , 2Ax , 3Sx",55.22,3,62.41,2,-1.00116.63,2006,olympics,True
2,3,Slutskaya,no_best,66.70,2.0,no_best,114.74,3.0,no_best,181.44,...,🇷🇺,"3Lz , 3S+2T+2Lo , 3F , 2F+2Tx , 3Lox , 3T+2Tx ...",53.87,5,61.87,3,-1.00114.74,2006,olympics,True
3,4,Suguri,SB,61.75,4.0,SB,113.48,4.0,SB,175.23,...,🇯🇵,"3Lz+2T , 3F , 3S , 3Lzx , 2F+2Tx , 3Tx , 2A+2Tx",54.23,4,59.25,4,-0.00113.48,2006,olympics,True
4,5,Rochette,no_best,55.85,9.0,SB,111.42,5.0,SB,167.27,...,🇨🇦,"3F , 3Lz+2T , 3Lo , 3S , 2Lzx , 3T+2T+2Lox , 2...",55.29,2,56.13,5,-0.00111.42,2006,olympics,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1850,3,You,SB,70.73,5.0,SB,146.24,2.0,SB,216.97,...,🇰🇷,"3Aq , 3Lz+3Tq , 3Lo , 2A , 3Lz+1Eu+3Sx , 2A+3T...",77.09,2,69.15,3,-0.00146.24,2022,gp_usa,True
1851,4,Sakamoto,SB,71.16,4.0,SB,144.77,3.0,SB,215.93,...,🇯🇵,"2A , 3Lz! , 3F+2T , 3S , 3F+3Tx , 2A+3T+2Tx , ...",74.54,3,70.23,1,-0.00144.77,2022,gp_usa,True
1854,7,Miyahara,SB,66.36,8.0,SB,134.15,6.0,SB,200.51,...,🇯🇵,"3Lz<+3T< , 2A , 3Lo , 3S , 3F!<+2T+2Lox , 3Lzx...",65.24,8,68.91,5,-0.00134.15,2022,gp_usa,True
1855,8,Kim,SB,70.56,6.0,SB,128.78,8.0,SB,199.34,...,🇰🇷,"3Lzq , 2A+3T , 3Lo , 3F , 3Lz+2Tx , 3S+2T+2Lox...",64.91,9,64.87,7,-1.00128.78,2022,gp_usa,True


In [85]:
df_oly = df_oly.replace({'<<':'@'}, regex=True)

In [86]:
df_oly['free_elements'] = df_oly['free_elements'].replace({'x':'##'}, regex=True)

In [87]:
df_oly['deductions'] = df_oly['free_overall'].str[1:2]

In [88]:
df_oly['free_elements'] = df_oly['free_elements'].replace({'q':'~'}, regex=True)

In [89]:
df_oly.short_combo_jump.tail(20)

1814        3T+2T10
1815       3F<+2T11
1816        3Lz+3T1
1817         3F+3T2
1819     3Lz+1Lo*10
1826      3Lz+3Lo<4
1827        3Lz+3T1
1829         3F+3T3
1831        3Lz+3T2
1833        3Lz+2T6
1837         3F+3T2
1838       3Lz+3T<5
1840      3Lzq+3T<9
1847    2F!+COMBO12
1848        3Lz+3T1
1850        3Lz+3T4
1851        2F+3T10
1854      3Lzq+3T<9
1855        3Lz+3T4
1856      3Fq+3T<12
Name: short_combo_jump, dtype: object

In [134]:
df_oly['free_elements'] = df_oly['free_elements'].replace({'e':'$'}, regex=True)
df_oly['short_combo_jump'] = df_oly['short_combo_jump'].replace({'e':'$'}, regex=True)
df_oly['short_axel_jump'] = df_oly['short_axel_jump'].replace({'e':'$'}, regex=True)
df_oly['short_solo_jump'] = df_oly['short_solo_jump'].replace({'e':'$'}, regex=True)

In [96]:
df_oly.free_elements

0       3Lz+2Lo , 3S+2T , 3F , 2A , 3Lz## , 2Lo## , 3S...
1       3Lz , 3F , 3Lo , 3F+2T## , 3T+3S+SEQ## , 2A## ...
2       3Lz , 3S+2T+2Lo , 3F , 2F+2T## , 3Lo## , 3T+2T...
3       3Lz+2T , 3F , 3S , 3Lz## , 2F+2T## , 3T## , 2A...
4       3F , 3Lz+2T , 3Lo , 3S , 2Lz## , 3T+2T+2Lo## ,...
                              ...                        
1850    3A~ , 3Lz+3T~ , 3Lo , 2A , 3Lz+1Eu+3S## , 2A+3...
1851    2A , 3Lz! , 3F+2T , 3S , 3F+3T## , 2A+3T+2T## ...
1854    3Lz<+3T< , 2A , 3Lo , 3S , 3F!<+2T+2Lo## , 3Lz...
1855    3Lz~ , 2A+3T , 3Lo , 3F , 3Lz+2T## , 3S+2T+2Lo...
1856    3Lz~+1Eu+3F , 3Lz , 3T , 2A , 3S+2A+SEQ## , 3L...
Name: free_elements, Length: 1127, dtype: object

In [137]:
df_oly['under_rotations'] = df_oly.apply(lambda x: x.str.contains("\<"), axis=1).sum(axis=1)

df_oly['downgrades'] = df_oly.apply(lambda x: x.str.contains("\@"), axis=1).sum(axis=1)

df_oly['time_bonus_elements'] = df_oly.apply(lambda x: x.str.contains("\##"), axis=1).sum(axis=1)

df_oly['edge_error'] = df_oly.apply(lambda x: x.str.contains("\$"), axis=1).sum(axis=1)

df_oly['suspected_edge_error'] =  df_oly.apply(lambda x: x.str.contains("\!"), axis=1).sum(axis=1)

df_oly['suspected_rotation_error'] = df_oly.apply(lambda x: x.str.contains('\~'), axis=1).sum(axis=1)

df_oly['combo_no_credit'] =  df_oly.apply(lambda x: x.str.contains("\+COMBO"), axis=1).sum(axis=1)

df_oly['illegal_element'] =  df_oly.apply(lambda x: x.str.contains("\*"), axis=1).sum(axis=1)


In [141]:
df_oly['under_rotations'] = df_oly['under_rotations'].astype(int)
df_oly['downgrades'] = df_oly['downgrades'].astype(int)
df_oly['time_bonus_elements'] = df_oly['time_bonus_elements'].astype(int)
df_oly['edge_error'] = df_oly['edge_error'].astype(int)
df_oly['suspected_edge_error'] =  df_oly['suspected_edge_error'].astype(int)
df_oly['suspected_rotation_error'] = df_oly['suspected_rotation_error'].astype(int)
df_oly['combo_no_credit'] =  df_oly['combo_no_credit'].astype(int)
df_oly['illegal_element'] =  df_oly['illegal_element'].astype(int)
df_oly['deductions'] = df_oly['deductions'].astype(int)

In [142]:
df_oly['jump_errors'] = df_oly['under_rotations'] + df_oly['downgrades'] + df_oly['edge_error'] + df_oly['suspected_edge_error'] + df_oly['suspected_rotation_error'] + df_oly['combo_no_credit'] + df_oly['illegal_element'] + df_oly['deductions']

In [143]:
df_oly['jump_errors']

0       0
1       1
2       1
3       0
4       0
       ..
1850    2
1851    1
1854    4
1855    2
1856    2
Name: jump_errors, Length: 1127, dtype: int64

In [ ]:
train, test = train_test_split(df_oly, test_size=.2, random_state=123)

In [ ]:
train.event.value_counts()

In [ ]:
train.free_elements.value_counts()

In [ ]:
train.dtypes